# MWRR Activity
Integrantes: Marianne Trujillo ALtamirano, Ana Luisa Espinoza López.

In [5]:
!pip install dateparser

Defaulting to user installation because normal site-packages is not writeable


In [ ]:
#Importamos librerías
import pyxirr as px
import pandas as pd
import dateparser as dp

In [16]:
data = "data_actividad.xlsx"
balances = pd.read_excel(data, sheet_name='balances')
movements = pd.read_excel(data, sheet_name='movements')

In [66]:
# 1. Función limpieza balances
def clean_balance_data(data):
    """
    Devuelve un diccionario {contrato: df_balance}
    Cada df tiene columnas: balance_date, portfolio_value
    """
    cols = ["contract", "value_pos_mdo", "balance_date"]
    df = data[cols].copy()

    # Parse fechas
    df["balance_date"] = df["balance_date"].apply(lambda x: dp.parse(str(x)))

    # Diccionario de balances por contrato
    contract_dfs = {}
    for contract, group in df.groupby("contract"):
        grouped = (
            group.groupby("balance_date", as_index=False)["value_pos_mdo"]
            .sum()
            .rename(columns={"value_pos_mdo": "portfolio_value"})
            .sort_values("balance_date")
        )
        contract_dfs[contract] = grouped.reset_index(drop=True)

    return contract_dfs

In [67]:
bal_clean = clean_balance_data(balances)
bal_clean.keys()

dict_keys(['12861603', '20486403', 'AHA84901'])

#### Balance contrato 12861603

In [ ]:
bal_clean['12861603'].head()

,balance_date,portfolio_value
0,2023-01-12,100000.0000
1,2023-01-13,100000.0000
2,2023-01-16,267081.1981
3,2023-01-17,270193.5784
4,2023-01-18,270369.3147


#### Balance contrato 20486403

In [69]:
bal_clean['20486403'].head()

,balance_date,portfolio_value
0,2022-12-09,1.531486e+06
1,2022-12-13,1.532354e+06
2,2022-12-14,1.533558e+06
3,2022-12-15,1.533234e+06
4,2022-12-16,1.533722e+06


#### Balance contrato AHA84901

In [70]:
bal_clean['AHA84901'].head()

,balance_date,portfolio_value
0,2023-06-26,200000.0000
1,2023-06-27,204868.6095
2,2023-06-28,204914.2665
3,2023-06-29,204976.7575
4,2023-06-30,205024.9605


In [71]:
# 2. Función limpieza movimientos
def clean_movements_data(df_mov):
    """
    Devuelve un diccionario {contrato: df_mov}
    Cada df tiene columnas: description, movement_import, operation_date
    """
    cols = ["contract", "movement_import", "operation_date", "description"]
    df = df_mov[cols].copy()

     # Parse fechas
    df["operation_date"] = df["operation_date"].apply(
        lambda x: dp.parse(str(x)) if pd.notnull(x) else pd.NaT
    )
    
    # Clasificarion descripciones
    clasificacion = {
        "DEPOSITO DE EFECTIVO": "DEPOSITO DE EFECTIVO",
        "DEPOSITO DE EFECTIVO POR TRANSFERENCIA": "DEPOSITO DE EFECTIVO",
        "Compra en Reporto": "DEPOSITO DE EFECTIVO",
        "Compra Soc. de Inv.- Cliente": "DEPOSITO DE EFECTIVO",
        "RETIRO DE EFECTIVO": "RETIRO DE EFECTIVO",
        "Venta Normal": "RETIRO DE EFECTIVO",
        "Venta Soc. de Inv.- Cliente": "RETIRO DE EFECTIVO",
        "Vencimiento de Reporto": "RETIRO DE EFECTIVO",
        "Amortización (cliente)": "RETIRO DE EFECTIVO",
    }

    
    df["description_clean"] = df["description"].map(clasificacion)
    df = df[df["description_clean"].notna()]

    contract_dfs = {}
    for contract, group in df.groupby("contract"):
        # Seleccionar solo las columnas finales requeridas y ordenar por fecha
        movements_df = (group[["description_clean", "movement_import", "operation_date"]]
                       .copy()
                       .sort_values("operation_date"))
        contract_dfs[contract] = movements_df.reset_index(drop=True)

    return contract_dfs

In [72]:
mov_clean = clean_movements_data(movements)
mov_clean.keys()

dict_keys(['12861603', '20486403', 'AHA84901'])

#### Movimientos 12861603

In [56]:
mov_clean['12861603'].head()

,description_clean,movement_import,operation_date
0,DEPOSITO DE EFECTIVO,100000.0000,2023-01-12
1,DEPOSITO DE EFECTIVO,99915.0002,2023-01-13
2,RETIRO DE EFECTIVO,99996.1983,2023-01-16
3,DEPOSITO DE EFECTIVO,167000.0000,2023-01-16
4,DEPOSITO DE EFECTIVO,100008.7980,2023-01-16


#### Movimientos contrato 20486403

In [57]:
mov_clean['20486403'].head()

,description_clean,movement_import,operation_date
0,DEPOSITO DE EFECTIVO,13851.9599,2022-12-15
1,RETIRO DE EFECTIVO,13855.4502,2022-12-16
2,DEPOSITO DE EFECTIVO,13931.5475,2022-12-16
3,RETIRO DE EFECTIVO,13942.6592,2022-12-19
4,DEPOSITO DE EFECTIVO,13942.7990,2022-12-19


#### Movimientos contrato AHA84901

In [58]:
mov_clean['AHA84901'].head()

,description_clean,movement_import,operation_date
0,DEPOSITO DE EFECTIVO,200000.0000,2023-06-26
1,DEPOSITO DE EFECTIVO,199944.2358,2023-06-26
2,RETIRO DE EFECTIVO,199944.2358,2023-06-27
3,DEPOSITO DE EFECTIVO,199907.0719,2023-06-27
4,RETIRO DE EFECTIVO,199907.0719,2023-06-28


In [81]:

# 3. Función para agregar valores inicial y final
def add_initial_final_values(bal_clean, mov_clean):
    """
    Agrega valor inicial y final a los movimientos de cada contrato
    Devuelve un diccionario {contrato: df_mov_completo}
    """
    complete_data = {}
    
    for contract in mov_clean.keys():
        if contract not in bal_clean:
            continue
            
        # Obtener datos de movimientos y balances del contrato
        mov_df = mov_clean[contract].copy()
        bal_df = bal_clean[contract].copy()
        
        if len(bal_df) == 0 or len(mov_df) == 0:
            continue
        
        # Renombrar el primer movimiento como VALOR_INICIAL
        mov_df.iloc[0, mov_df.columns.get_loc('description_clean')] = 'VALOR_INICIAL'
        
        # Valor final (último balance) - signo positivo
        final_value = bal_df.iloc[-1]
        final_row = pd.DataFrame({
            'description_clean': ['VALOR_FINAL'],
            'movement_import': [final_value['portfolio_value']],
            'operation_date': [final_value['balance_date']]
        })
        
        # Ajustar signos de movimientos: depósitos negativos, retiros positivos
        mov_df_adjusted = mov_df.copy()
        for idx, row in mov_df_adjusted.iterrows():
            if row['description_clean'] == 'DEPOSITO DE EFECTIVO':
                mov_df_adjusted.at[idx, 'movement_import'] = -abs(row['movement_import'])
            elif row['description_clean'] == 'RETIRO DE EFECTIVO':
                mov_df_adjusted.at[idx, 'movement_import'] = abs(row['movement_import'])
            elif row['description_clean'] == 'VALOR_INICIAL':
                mov_df_adjusted.at[idx, 'movement_import'] = -abs(row['movement_import'])
        
        # Combinar todos los datos
        complete_df = pd.concat([mov_df_adjusted, final_row], ignore_index=True)
        complete_df = complete_df.sort_values('operation_date').reset_index(drop=True)
        
        complete_data[contract] = complete_df
    
    return complete_data

In [82]:
complete_data = add_initial_final_values(bal_clean, mov_clean)
complete_data.keys()

dict_keys(['12861603', '20486403', 'AHA84901'])

#### Flujos contrato 12861603

In [85]:
complete_data['12861603'].head()

,description_clean,movement_import,operation_date
0,VALOR_INICIAL,-100000.0000,2023-01-12
1,DEPOSITO DE EFECTIVO,-99915.0002,2023-01-13
2,RETIRO DE EFECTIVO,99996.1983,2023-01-16
3,DEPOSITO DE EFECTIVO,-167000.0000,2023-01-16
4,DEPOSITO DE EFECTIVO,-100008.7980,2023-01-16


In [94]:
complete_data['12861603'].tail(10)

,description_clean,movement_import,operation_date
439,RETIRO DE EFECTIVO,5177.6280,2023-11-17
440,DEPOSITO DE EFECTIVO,-100.0667,2023-11-17
441,RETIRO DE EFECTIVO,100.1835,2023-11-21
442,DEPOSITO DE EFECTIVO,-3002.8178,2023-11-30
443,VALOR_FINAL,223059.8805,2023-11-30
444,RETIRO DE EFECTIVO,30000.0000,2023-11-30
445,RETIRO DE EFECTIVO,17847.3064,2023-11-30
446,DEPOSITO DE EFECTIVO,-3003.6992,2023-12-01
447,RETIRO DE EFECTIVO,3003.6938,2023-12-01
448,RETIRO DE EFECTIVO,3006.3280,2023-12-04


#### Flujos contrato 20486403

In [89]:
complete_data['20486403'].head()

,description_clean,movement_import,operation_date
0,VALOR_INICIAL,-13851.9599,2022-12-15
1,RETIRO DE EFECTIVO,13855.4502,2022-12-16
2,DEPOSITO DE EFECTIVO,-13931.5475,2022-12-16
3,RETIRO DE EFECTIVO,13942.6592,2022-12-19
4,DEPOSITO DE EFECTIVO,-13942.7990,2022-12-19


In [93]:
complete_data['12861603'].tail(10)

,description_clean,movement_import,operation_date
439,RETIRO DE EFECTIVO,5177.6280,2023-11-17
440,DEPOSITO DE EFECTIVO,-100.0667,2023-11-17
441,RETIRO DE EFECTIVO,100.1835,2023-11-21
442,DEPOSITO DE EFECTIVO,-3002.8178,2023-11-30
443,VALOR_FINAL,223059.8805,2023-11-30
444,RETIRO DE EFECTIVO,30000.0000,2023-11-30
445,RETIRO DE EFECTIVO,17847.3064,2023-11-30
446,DEPOSITO DE EFECTIVO,-3003.6992,2023-12-01
447,RETIRO DE EFECTIVO,3003.6938,2023-12-01
448,RETIRO DE EFECTIVO,3006.3280,2023-12-04


#### Flujos contrato AHA84901

In [92]:
complete_data['AHA84901'].head()

,description_clean,movement_import,operation_date
0,VALOR_INICIAL,-200000.0000,2023-06-26
1,DEPOSITO DE EFECTIVO,-199944.2358,2023-06-26
2,RETIRO DE EFECTIVO,199944.2358,2023-06-27
3,DEPOSITO DE EFECTIVO,-199907.0719,2023-06-27
4,DEPOSITO DE EFECTIVO,-5000.0000,2023-06-27


In [91]:
complete_data['AHA84901'].tail(10)

,description_clean,movement_import,operation_date
225,RETIRO DE EFECTIVO,11880.2954,2023-11-28
226,DEPOSITO DE EFECTIVO,-11959.7666,2023-11-28
227,DEPOSITO DE EFECTIVO,-11935.0192,2023-11-29
228,RETIRO DE EFECTIVO,11963.2556,2023-11-29
229,DEPOSITO DE EFECTIVO,-11936.1090,2023-11-30
230,VALOR_FINAL,430842.6591,2023-11-30
231,RETIRO DE EFECTIVO,11938.1694,2023-11-30
232,DEPOSITO DE EFECTIVO,-11939.6496,2023-12-01
233,RETIRO DE EFECTIVO,11939.5910,2023-12-01
234,RETIRO DE EFECTIVO,11950.0988,2023-12-04


In [86]:
# 4. Función para calcular MWRR
def calculate_mwrr_by_contract(complete_data):
    """
    Calcula el MWRR (TIR) para cada contrato usando pyxirr
    Devuelve un diccionario {contrato: mwrr}
    """
    mwrr_results = {}
    
    for contract, df in complete_data.items():
        try:
            dates = df['operation_date'].tolist()
            amounts = df['movement_import'].tolist()
            
            # Calcular MWRR usando pyxirr
            mwrr = px.xirr(dates, amounts)
            mwrr_results[contract] = mwrr
            
        except Exception as e:
            print(f"Error calculando MWRR para contrato {contract}: {e}")
            mwrr_results[contract] = None
    
    return mwrr_results

#### Resultados

In [95]:
mwrr_by_contract = calculate_mwrr_by_contract(complete_data)

print("MWRR por contrato:")
for contract, mwrr in mwrr_by_contract.items():
    if mwrr is not None:
        print(f"{contract}: {mwrr:.2%}")
    else:
        print(f"{contract}: Error en cálculo")

MWRR por contrato:
12861603: 53.40%
20486403: 13081.19%
AHA84901: 10.58%
